In [1]:
import os
os.chdir("../captain-project-main")

In [2]:
import captain as cn

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


# Examine how list_cell looks like

### list_cell is a list of cell objects that store the information like carrying capacity and species_hist which is a list of numbers meaning the abundance of each species. It is created by function `cn.init_cell_objects` and updated by `cn.init_simulated_system`

In [3]:
import numpy as np
import random
import matplotlib.pyplot as plt

# matplotlib.use('Agg')
import matplotlib.backends.backend_pdf

plt.rcParams.update({"figure.max_open_warning": 0})
import seaborn as sns

sns.set()
import pickle
import sys, os

In [4]:
seed=0
disp_rate=0.3
grid_size=50
n_species=100
cell_capacity=100
out_dir="./sim_data"
verbose=1

In [5]:
if seed == 0:
        rseed = np.random.randint(1000, 9999)
else:
    rseed = seed
np.random.seed(rseed)
random.seed(rseed)

out_tag = ""


In [6]:

# init Area
cell_carrying_capacity = cell_capacity  # init (max) individuals per cell
total_cells = grid_size ** 2
SP_ID = np.arange(n_species)
death_at_climate_boundary = 0.25  # if set to 0.5 death probability is 50% at the
# climatic boundaries (based on empirical init range)
lat_steepness = 0.1  # 5 degrees difference

out_tag += "_d%s_t%s" % (disp_rate, death_at_climate_boundary)

# rel_rank_abundance_distribution = np.sort(np.random.weibull(0.75,n_species))[::-1] +(np.random.uniform(0.2,0.5,n_species))
# TRUNCATED WEIBULL
# represent the distribution of species abundances
#rare species are more common, abundant species are less common
# array([8.043, 6.993, 4.789, 4.553, 4.466, 4.034, 4.015, 3.839, 3.704,
#        3.66 , 3.576, 2.866, 2.633, 2.481, 2.348, 2.344, 2.313, 2.241,
#        2.01 , 1.954, 1.836, 1.797, 1.776, 1.628, 1.524, 1.51 , 1.502,
#        1.489, 1.352, 1.346, 1.337, 1.335, 1.325, 1.315, 1.293, 1.29 ,
#        1.277, 1.266, 1.189, 1.174, 1.132, 1.126, 1.113, 1.085, 1.056,
#        1.021, 1.014, 0.964, 0.91 , 0.866, 0.862, 0.854, 0.852, 0.852,
#        0.79 , 0.779, 0.775, 0.76 , 0.713, 0.668, 0.652, 0.651, 0.645,
#        0.603, 0.593, 0.587, 0.533, 0.53 , 0.526, 0.483, 0.465, 0.464,
#        0.454, 0.44 , 0.433, 0.401, 0.38 , 0.349, 0.347, 0.339, 0.337,
#        0.33 , 0.324, 0.307, 0.273, 0.216, 0.202, 0.189, 0.183, 0.182,
#        0.166, 0.146, 0.142, 0.14 , 0.139, 0.132, 0.131, 0.128, 0.105,
#        0.081])
rel_rank_abundance_distribution = np.sort(
    np.random.weibull(0.75, n_species + int(0.10 * n_species))
)[::-1]
rel_rank_abundance_distribution = rel_rank_abundance_distribution[0:n_species]
# fatter tail:
# rel_rank_abundance_distribution = np.sort(np.random.weibull(0.5,n_species))[::-1]+np.sort(np.random.uniform(0.2,0.5,n_species))

# get distances all vs all (3D array)
coord = np.linspace(0.5, grid_size - 0.5, grid_size)
d_matrix = cn.get_all_to_all_dist_jit(coord, grid_size)
# table with cell ID and x/y coordinantes
cell_id_n_coord = cn.get_coordinates_jit(coord, grid_size).astype(int)
#array([
# [0, 0, 0],
# [1, 0, 1],
# [2, 0, 2],
# [3, 1, 0],
# [4, 1, 1],
# [5, 1, 2],
# [6, 2, 0],
# [7, 2, 1],
# [8, 2, 2]
# ])
cell_file_pkl = "pickles/init_cell_%s_c%s_s%s%s.pkl" % (
    rseed,
    grid_size,
    n_species,
    out_tag,
)
cell_file_pkl = os.path.join(out_dir, cell_file_pkl)



calculating  distances...
done.


In [7]:
# init cell object
list_cells = cn.init_cell_objects(
    cell_id_n_coord, d_matrix, n_species, cell_carrying_capacity, lat_steepness
)
# climatic gradient
temp_by_cell = np.array([c.temperature for c in list_cells]).reshape(
    grid_size, grid_size
)

# tot number of individuals per species (init)
tot_init_individuals = cell_carrying_capacity * total_cells
# the chance of each species being selected is proportional to its relative abundance
rel_freq_species = rel_rank_abundance_distribution / np.sum(
    rel_rank_abundance_distribution
)
#sample number of total individuals times based on the relative abundance, len(sample_individuals_per_species) = tot_init_individuals
sample_individuals_per_species = np.random.choice(
    SP_ID, size=tot_init_individuals, p=rel_freq_species, replace=True
)
# count number of individuals per species, len(n_individuals_per_species) = n_species
n_individuals_per_species = np.unique(
    sample_individuals_per_species, return_counts=True
)

# init all species
# randomize species order, len(rnd_species_order) = n_species
rnd_species_order = np.random.choice(SP_ID, len(SP_ID), replace=False)
#if there is available space in the cells
aval_space = np.array([c.room_for_one for c in list_cells])
#number of individuals per cell
n_indviduals_per_cell_1D = np.array([c.n_individuals for c in list_cells])


init cells...
done.


In [31]:
n_individuals_per_species

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
        85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
 array([15233, 13148,  9066,  8504,  8414,  7625,  7500,  7148,  7154,
         6802,  6692,  5429,  4979,  4643,  4459,  4352,  4223,  4195,
         3776,  3661,  3445,  3337,  3273,  3070,  2804,  2853,  2820,
         2766,  2591,  2502,  2502,  2438,  2474,  2455,  2349,  2418,
         2363,  2360,  2228,  2177,  2120,  2067,  2093,  2102,  1891,
         1904,  1902,  1777,  1727,  1631,  1569,  1644,  1587,  1624,
         1486,  1425,  1394,  1506,  1325,  1286,  1308,  1164,  1175,
         1122,  1135,  1133,   999,   935,   967,   9

In [32]:
cell_carrying_capacity

100

In [33]:
j = 1
for species_id in rnd_species_order:
    # get the maximum number of individuals for this species can reach
    max_n_ind = n_individuals_per_species[1][species_id]
    if verbose:
        cn.print_update(
            "%s/%s init species %s (%s ind.) %s"
            % (j, n_species, species_id, max_n_ind, cn.get_nature_emoji())
        )
    # check if there is available space in the cells, if no 1->0
    aval_space[n_indviduals_per_cell_1D == cell_carrying_capacity] = 0
    #pick a random cell to start the species
    rnd_starting_cell = np.random.choice(
        cell_id_n_coord[:, 0], p=aval_space / np.sum(aval_space)
    )
    n_indviduals_per_cell_1D, n_indviduals_per_cell_sp_i = cn.init_propagate_species(
        [rnd_starting_cell],
        max_n_ind,
        list_cells,
        cell_id_n_coord,
        cell_carrying_capacity,
        species_id,
        disp_rate,
    )

    j += 1

100/100 init species 50 (1569 ind.) 🐊

In [ ]:
rnd_starting_cell

[1199]

# Examine how disturbance looks like

In [15]:
cn.get_disturbance

<function captain.biodivsim.DisturbanceGenerator.get_disturbance(mode, seed=0)>

# Examine how to set up the environment

In [17]:
cn.init_simulated_system

<function captain.biodivinit.SimulatorInit.init_simulated_system(seed=0, disp_rate=0.3, grid_size=50, n_species=100, cell_capacity=100, out_dir='./sim_data', verbose=1)>

In [6]:
sim_file = cn.init_simulated_system(n_species=25,
                                    grid_size=100,
                                    cell_capacity=25,
                                    out_dir='./sim_data')

calculating  distances...
done.
init cells...
done.
25/25 init species 2 (25092 ind.) 🌴
System saved in:  ./sim_data/pickles/init_cell_5370_c100_s25_d0.3_t0.25.pkl


In [7]:
gridinit = cn.PickleInitializer(sim_file)

In [8]:
h = gridinit.getInitialState()

In [9]:
#examine the shape
h.shape

(25, 100, 100)

In [12]:
np.einsum("xyz -> yz", h).shape

(100, 100)

In [4]:
env = cn.simulate_biodiv_env(sim_file,
                             dispersal_rate=0.3,
                             climate_mode=3,
                             disturbance_mode=8)

runnerInput.runMode 0
Simulating 25 species tree...
re-loading grid...
Rnd sens [0.947 0.214 0.511 0.849 0.071]
Rnd climate sens [0.416 0.459 0.414 0.607 0.244]
Simulating 25 species tree...
Rnd sp values:  [0.03  6.337 0.096 0.034 0.014]
